## Module Import 

In [ ]:
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
import h5py
import re
import os
import time as t
import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation
from tensorflow.python.keras.layers import Dense
np.set_printoptions(precision=8)
seed=99

print(tf. __version__)

2.7.0


In [ ]:
# change the directory to data location
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/PINN')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read the file
file = 'molar_fraction_major_chemical_marker.hdf5'
f = h5py.File(file, 'r')

In [ ]:
# Get the feature and the label
# The dataset have larger points HHR region [data= Augumentation]
dataset = np.array(f.get('molar_fraction'))
print(dataset.shape)

(50901851, 6)


In [ ]:
# Separate the feature and the label
X_old = dataset[:,:-1]
Y = dataset[:,[-1]]
print(X_old.shape,Y.shape)

(50901851, 5) (50901851, 1)


In [ ]:
# the product of mass fraction and density to find mole fraction.
# get the mass fraction
y_nh = X_old[:,0:1]
y_nh2 = X_old[:,1:2]
y_oh = X_old[:,2:3]
# Get the tmperature{The additional input}
temp = X_old[:,4:5]
# get the density
density = X_old[:,3:4]

# A linear relation between mass fractio and mole fraction
X_nh = y_nh *density
X_nh2 = y_nh2 *density
X_oh = y_oh *density

# The new input
X = np.c_[X_nh,X_nh2,X_oh,temp]
print(X)

[[3.24108411e-07 7.15551340e-06 1.50022992e-04 1.88831462e+03]
 [3.24264966e-07 7.15909405e-06 1.50018731e-04 1.88830965e+03]
 [3.24387130e-07 7.16189148e-06 1.50015388e-04 1.88830574e+03]
 ...
 [1.03291829e-05 3.26863957e-05 8.84004547e-04 2.29063320e+03]
 [1.01743229e-05 3.21374957e-05 8.88756537e-04 2.29142432e+03]
 [9.85278722e-06 3.10329526e-05 8.92953618e-04 2.29242941e+03]]


In [ ]:
# change the datattype
X = np.float32(X)
Y = np.float32(Y)


In [ ]:
#shuffle the dataset
X,Y = shuffle(X, Y, random_state=seed)
print(X, Y)

[[1.1436125e-05 6.6332905e-05 9.4569393e-04 1.8394366e+03]
 [2.3975999e-05 3.6371115e-04 1.9834892e-04 1.5187620e+03]
 [5.1061328e-05 5.8826891e-04 1.4799622e-04 1.7107389e+03]
 ...
 [3.0624233e-05 4.4022195e-04 2.0987148e-04 1.5530289e+03]
 [4.0838397e-05 2.8424856e-04 6.6376093e-04 1.8717084e+03]
 [3.5612813e-06 1.4560942e-04 2.2836184e-05 1.4690447e+03]] [[2.6408230e+09]
 [4.5877038e+09]
 [8.5497395e+09]
 ...
 [6.0870994e+09]
 [8.8248003e+09]
 [2.3323902e+08]]


In [ ]:
# Normalization
epsilon = 1e-10
Y_n = (Y - np.min(Y) + epsilon  *(np.max(Y)-np.min(Y)))/ (np.max(Y)-np.min(Y) + epsilon   * (np.max(Y)-np.min(Y)))
X_n = (X - np.min(X, axis=0) + epsilon  *(np.max(X, axis=0)-np.min(X, axis=0)))/ (np.max(X, axis=0)-np.min(X, axis=0) + epsilon  * (np.max(X, axis=0)-np.min(X, axis=0)))

In [ ]:
# # create the normalization fit 
# scale_X = MinMaxScaler()
# scale_Y = MinMaxScaler()

# # The fit function
# print(scale_X.fit(X))
# print(scale_Y.fit(Y))
print(X_n,Y_n)

[[0.12565005 0.0882831  0.546299   0.58453494]
 [0.2634644  0.48807305 0.11424741 0.37687802]
 [0.5611346  0.78996444 0.08513775 0.5011951 ]
 ...
 [0.33652908 0.5909328  0.12090877 0.39906803]
 [0.44878367 0.381245   0.3833092  0.6054329 ]
 [0.0391048  0.19486102 0.01278082 0.34468296]] [[0.13353658]
 [0.23251079]
 [0.43393013]
 ...
 [0.30873606]
 [0.44791347]
 [0.01114142]]


In [ ]:
#test_train split 
X_train, X_test, Y_train, Y_test = train_test_split(X_n, Y_n, test_size=0.1)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(45811665, 4) (5090186, 4) (45811665, 1) (5090186, 1)


In [ ]:
# # normalize the train data and label
# X_train_n = scale_X.transform(X_train)
# Y_train_n = scale_Y.transform(Y_train)
# print(X_train_n)
# print(Y_train_n)

In [ ]:
# Create the chunk 
X_chunk = tf.Variable(X_train)
Y_chunk = tf.Variable(Y_train)
print(X_chunk, Y_chunk)

<tf.Variable 'Variable:0' shape=(45811665, 4) dtype=float32, numpy=
array([[0.0233122 , 0.16263473, 0.00569963, 0.20774056],
       [0.52587634, 0.63037336, 0.08785675, 0.61297464],
       [0.11136776, 0.46825173, 0.00814626, 0.26391652],
       ...,
       [0.5899217 , 0.6467477 , 0.22106957, 0.5422071 ],
       [0.4257508 , 0.25201622, 0.19658317, 0.8839737 ],
       [0.08465935, 0.3928106 , 0.01286309, 0.13305877]], dtype=float32)> <tf.Variable 'Variable:0' shape=(45811665, 1) dtype=float32, numpy=
array([[0.00930248],
       [0.2805789 ],
       [0.06084422],
       ...,
       [0.5775659 ],
       [0.11447497],
       [0.07792261]], dtype=float32)>


In [ ]:
# Creaate the batch slices.
train_data = tf.data.Dataset.from_tensor_slices((X_chunk, Y_chunk))
train_data = train_data.batch(2048, drop_remainder=True)
print(train_data)
# test_data = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
# test_data = test_data.batch(1024, drop_remainder=True)

<BatchDataset shapes: ((2048, 4), (2048, 1)), types: (tf.float32, tf.float32)>


In [ ]:
# Get the pretrained model
# Read the file
file = 'model_with_molar_fraction_major_CM.h5'
model = tf.keras.models.load_model(file)

# Get summary
model.summary()

Model: "Heat_release_rate"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 h_layer1 (Dense)            (None, 40)                200       
                                                                 
 h_layer2 (Dense)            (None, 24)                984       
                                                                 
 h_layer3 (Dense)            (None, 24)                600       
                                                                 
 output_layers (Dense)       (None, 1)                 25        
                                                                 
Total params: 1,809
Trainable params: 1,809
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# The Ki the is respective exponents of the species massfraction and temperature, which could yeild us a perfect reconstruction of the HRR
k1,k2,k3,k4,k5 = tf.Variable(-1.8),tf.Variable(-1.6),tf.Variable(-1.4),tf.Variable(-1.5),tf.Variable(1.3)

In [ ]:
# The customized mean square error, taking the batch data at any instance
def customized_MSE(true, pred):
    MSE = tf.reduce_mean(tf.square(true-pred))
    return MSE
# The customized loss taking the non-linear equation as residual
def total_mse(MSE_NN, x, y, k1, k2, k3, k4, k5):# the MSE_NN is the mean squared error of the neural network 
    residual = tf.square(tf.math.log(y) - (k1 * tf.math.log(x[:,0:1]) + k2 * tf.math.log(x[:,1:2]) + k3 * tf.math.log(x[:,2:3]) + k4 * tf.math.log(x[:,3:4]) + tf.math.log(k5)))
    MSE_func = tf.reduce_mean(residual)
    total =  MSE_NN + MSE_func
    return total

In [ ]:
# The optimizer
# optimizer= tf.keras.optimizers.SGD(learning_rate=0.3,momentum=0.8)
optimizers = tf.keras.optimizers.Adam(
    learning_rate=0.03, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')

In [ ]:
#Training with customized optimization technic (Automatic-Gradient)
epochs = 25
#lr_rate = 0.03

for epoch in range(epochs):
  print("Starting a new epoch",epoch+1)
  for step, (X_batch, Y_batch) in enumerate(train_data):
    with tf.GradientTape(persistent=True) as tape:
      Y_pred = model(X_batch)
      current_loss = customized_MSE(Y_batch,Y_pred)
      new_loss = total_mse(current_loss, X_batch, Y_batch, k1, k2, k3, k4, k5)

    # The gradient check  
    gradient = tape.gradient(new_loss, [k1,k2,k3,k4,k5])
    grad = tape.gradient(current_loss, model.trainable_variables)

    #Assign new variables to the model using optimizer isntead of sub assign
    optimizers.apply_gradients(zip(grad, model.trainable_variables))
    optimizers.apply_gradients(zip(gradient, [k1,k2,k3,k4,k5]))
  
  print("k1:{} k2:{} k3:{} k4:{} k5:{}".format(k1.numpy(),k2.numpy(),k3.numpy(),k4.numpy(),k5.numpy()))
  print("current_Loss: {} new_loss: {}".format(current_loss.numpy(), new_loss.numpy()))  

Starting a new epoch 1
k1:0.9253278374671936 k2:0.17793548107147217 k3:0.6602041125297546 k4:-1.5913439989089966 k5:0.9866951107978821
current_Loss: 6.717333599226549e-05 new_loss: 0.15743225812911987
Starting a new epoch 2
k1:0.9253277778625488 k2:0.17793557047843933 k3:0.6602041125297546 k4:-1.591343641281128 k5:0.9866951704025269
current_Loss: 0.00010370668314862996 new_loss: 0.15746881067752838
Starting a new epoch 3
k1:0.9253278970718384 k2:0.17793549597263336 k3:0.6602041721343994 k4:-1.5913439989089966 k5:0.9866951704025269
current_Loss: 0.00011268029629718512 new_loss: 0.15747776627540588
Starting a new epoch 4
k1:0.9253278970718384 k2:0.17793545126914978 k3:0.6602041721343994 k4:-1.591343879699707 k5:0.9866951704025269
current_Loss: 8.474945207126439e-05 new_loss: 0.1574498564004898
Starting a new epoch 5
k1:0.925327718257904 k2:0.17793552577495575 k3:0.6602041125297546 k4:-1.591343641281128 k5:0.9866951107978821
current_Loss: 4.0901351894717664e-05 new_loss: 0.157405987381935

In [ ]:
# #Training with customized optimization technic (Automatic-Gradient)
# epochs = 25
# lr_rate = 0.03

# for epoch in range(epochs):
#   print("Starting a new epoch",epoch+1)
#   for step, (X_batch, Y_batch) in enumerate(train_data):
#     with tf.GradientTape(persistent=True) as tape:
#       Y_pred = model(X_batch)
#       current_loss = customized_MSE(Y_batch,Y_pred)
#       new_loss = total_mse(current_loss, X_batch, Y_batch, k1, k2, k3, k4, k5)

#     # The gradient check  
#     gradient = tape.gradient(new_loss, [k1,k2,k3,k4,k5])
#     grad = tape.gradient(current_loss, model.trainable_variables)

#     #Assign new variables to the model using optimizer isntead of sub assign
#     optimizer.apply_gradients(zip(grad, model.trainable_variables))

#     #Update the Ki to reduce the residual
#     k1.assign_sub(gradient[0]*lr_rate)
#     k2.assign_sub(gradient[1]*lr_rate)
#     k3.assign_sub(gradient[2]*lr_rate)
#     k4.assign_sub(gradient[3]*lr_rate)
#     k5.assign_sub(gradient[4]*lr_rate)
  
#   print("k1:{} k2:{} k3:{} k4:{} k5:{}".format(k1.numpy(),k2.numpy(),k3.numpy(),k4.numpy(), k5.numpy()))
#   print("current_Loss: {} new_loss: {}".format(current_loss.numpy(), new_loss.numpy()))  

Starting a new epoch 1
k1:0.9110038876533508 k2:0.18080781400203705 k3:0.6620661020278931 k4:-1.582798719406128 k5:0.9866163730621338
current_Loss: 3.9711529097985476e-05 new_loss: 0.15844884514808655
Starting a new epoch 2
k1:0.9110038876533508 k2:0.18080781400203705 k3:0.6620661020278931 k4:-1.582798719406128 k5:0.9866163730621338
current_Loss: 3.971100159105845e-05 new_loss: 0.15844884514808655
Starting a new epoch 3
k1:0.9110038876533508 k2:0.18080781400203705 k3:0.6620661020278931 k4:-1.582798719406128 k5:0.9866163730621338
current_Loss: 3.97089243051596e-05 new_loss: 0.15844884514808655
Starting a new epoch 4
k1:0.9110038876533508 k2:0.18080781400203705 k3:0.6620661020278931 k4:-1.582798719406128 k5:0.9866163730621338
current_Loss: 3.970593388658017e-05 new_loss: 0.15844884514808655
Starting a new epoch 5
k1:0.9110038876533508 k2:0.18080781400203705 k3:0.6620661020278931 k4:-1.582798719406128 k5:0.9866163730621338
current_Loss: 3.970246689277701e-05 new_loss: 0.15844883024692535
